In [1]:
import sys
sys.path.append( "/home/mathia/PycharmProjects/neslearn-git")
sys.path.append( "/home/mathia/PycharmProjects/neslearn-git/extern/nes-async-rl/")

In [2]:
import pandas
import hsa.ba.rewards as rewards
import itertools
from extern.fceux_learningenv.nes_python_interface.nes_python_interface import NESInterface
import hsa.machine_constants
from hsa.nes_python_input import py_to_nes_wrapper

import numpy as np
import matplotlib.pyplot as plt
import itertools
import os.path

In [3]:
from demo_a3c_nes import eval_performance as org_eval_performance

In [4]:
from dqn_phi import dqn_phi
import nes 
from a3c_nes import A3CLSTM
import chainer
from chainer import serializers

In [5]:
example_model_path = "/home/mathia/checkpoints/nes_a3c_scaled_finer_main_reward_1/16000000_finish.h5"
rom_path = "/home/mathia/mario.nes"
nr_of_attempts = 30

In [6]:
nesApi = NESInterface(rom_path)
nesApi.render()
nesApi.reset_game()

In [7]:
base_paht = "/home/mathia/checkpoints/{}/{}"
model_names = [
    "scaled_fine_main_reward_1" # coars
    ,"nes_a3c_4" # ehrenbrav
    ,"nes_a3c_scaled_finer_main_reward_1", # main
     "nes_a3c_finer_main_reward_1" # small
    ,"scaled_finer_main_reward_with_points_1" #with points
]

model_chepoint_combos = [(model_name,"16000000_finish.h5")for model_name in model_names]
# model_chepoint_combos.append(("nes_a3c_4","9000000.h5"))
# model_chepoint_combos.append(("scaled_fine_main_reward_1","9400000.h5"))
# model_chepoint_combos.append(("nes_a3c_scaled_finer_main_reward_1","9200000.h5"))
#model_chepoint_combos = itertools.product(model_names, chepoint_filenames) 
model_snapshots = {(directory,checkpoint):base_paht.format(directory,checkpoint) 
                   for (directory,checkpoint) 
                   in model_chepoint_combos}
# make sure every file exists
for path in model_snapshots.values():
    if not os.path.isfile(path):
        print(path)

In [8]:
model_snapshots

{('nes_a3c_4',
  '16000000_finish.h5'): '/home/mathia/checkpoints/nes_a3c_4/16000000_finish.h5',
 ('nes_a3c_finer_main_reward_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/nes_a3c_finer_main_reward_1/16000000_finish.h5',
 ('nes_a3c_scaled_finer_main_reward_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/nes_a3c_scaled_finer_main_reward_1/16000000_finish.h5',
 ('scaled_fine_main_reward_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/scaled_fine_main_reward_1/16000000_finish.h5',
 ('scaled_finer_main_reward_with_points_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/scaled_finer_main_reward_with_points_1/16000000_finish.h5'}

In [9]:
#n_actions = nes.NES(hsa.machine_constants.mario_rom_location).number_of_actions

def load_model(weights_path):
    n_actions = nes.n_actions
    model = A3CLSTM(n_actions)
    serializers.load_hdf5(weights_path, model)
    return model
example_model = load_model(example_model_path)

In [10]:
def make_eval_functions():
    return {
        "main_reward": rewards.make_scaled_finer_main_reward(),
        "points": rewards.make_delta_points(),
        "time_left": rewards.reward_for_time_left
    }

In [18]:
def close_eval_performance(env, model, deterministic=False,
                     record_screen_path=None):
    #env = nes.NES(rom,record_screen_path=record_screen_path)
    model.reset_state()
    test_r = 0
    while not env.is_terminal:
        s = chainer.Variable(np.expand_dims(dqn_phi(env.state), 0))
        pout = model.pi_and_v(s)[0]
        model.unchain_backward()
        if deterministic:
            a = pout.most_probable_actions[0]
        else:
            a = pout.action_indices[0]
        test_r += env.receive_action(a)
    return test_r

def wrapped_eval_performance(model):
    other_rewards = make_eval_functions()
    sums = {name:0 for name in other_rewards.keys()}
    def action_callback(frame_skip_i, ram):
        nonlocal sums
        for name, reward_function in other_rewards.items():
            sums[name] += reward_function(ram)
    eherenbrav_sum = close_eval_performance(
        nes.NES(rom_path, outside_nes_interface=nesApi,
                every_action_callback=action_callback)
        , model)
    sums["eherenbrav"] = eherenbrav_sum
    return sums

In [20]:
eval_results = {}
for (experiment_name,model_name), weights in model_snapshots.items():
    model = load_model(weights)
    for attempt_nr in range(nr_of_attempts):
        nesApi.reset_game()
        env = nes.NES(rom_path,outside_nes_interface=nesApi,frame_skip=1)
        performance = wrapped_eval_performance(model)
        print((experiment_name,model_name,attempt_nr), performance)
        eval_results[(experiment_name,model_name,attempt_nr)] = performance

('nes_a3c_4', '16000000_finish.h5', 0) {'eherenbrav': 11075, 'points': 40.0, 'time_left': 0, 'main_reward': 26.3671875}


('nes_a3c_4', '16000000_finish.h5', 1) {'eherenbrav': 10060, 'points': 30.0, 'time_left': 0, 'main_reward': 60.625}


('nes_a3c_4', '16000000_finish.h5', 2) {'eherenbrav': 13255, 'points': 40.0, 'time_left': 0, 'main_reward': 61.0546875}


('nes_a3c_4', '16000000_finish.h5', 3) {'eherenbrav': 13175, 'points': 60.0, 'time_left': 0, 'main_reward': 49.375}


('nes_a3c_4', '16000000_finish.h5', 4) {'eherenbrav': 14000, 'points': 60.0, 'time_left': 0, 'main_reward': 48.90625}


...

('nes_a3c_finer_main_reward_1', '16000000_finish.h5', 25) {'eherenbrav': 3490, 'points': 40.0, 'time_left': 0, 'main_reward': 11.3671875}


('nes_a3c_finer_main_reward_1', '16000000_finish.h5', 26) {'eherenbrav': 5810, 'points': 50.0, 'time_left': 0, 'main_reward': 11.0546875}


('nes_a3c_finer_main_reward_1', '16000000_finish.h5', 27) {'eherenbrav': 10140, 'points': 10.0, 'time_left': 0, 'main_reward': 27.8125}


('nes_a3c_finer_main_reward_1', '16000000_finish.h5', 28) {'eherenbrav': 10340, 'points': 30.0, 'time_left': 0, 'main_reward': 28.203125}


('nes_a3c_finer_main_reward_1', '16000000_finish.h5', 29) {'eherenbrav': 9365, 'points': 170.0, 'time_left': 0, 'main_reward': 8.7890625}


In [21]:
result_df = pandas.DataFrame.from_dict(eval_results,orient="index")

In [25]:
result_df.to_csv("eval_results_6.csv")

In [26]:
result_df.to_pickle("eval_results_6.pickle")

In [19]:
result_df.groupby?

# ladida just rewiting time left

In [10]:
def make_time_left():
    counted_once = False
    def time_left(ram):
        nonlocal counted_once
        if counted_once:
            return 0
        if ram[0x001D] == 3:
            counted_once = True
            return rewards.time_left(ram) 
        return 0
    return  time_left

#### check close eval performance

In [11]:
#,frame_skip=1
close_eval_performance(nes.NES(rom_path,outside_nes_interface=nesApi),example_model)

43775

### Trying it with a callback

In [16]:
wrapped_eval_performance(example_model)

{'eherenbrav': 41990,
 'main_reward': 163.47812500000001,
 'points': 2095.0,
 'time_left': 359}